# Part II. Simplified DES

In this section, you will implement a simplified version of the DES block cipher algorithm. Naturally
enough, it is called SDES, and it is designed to have the features of the DES algorithm but scaled down
so it is more tractable to understand. (Note however, that SDES is in no way secure and should not
be used for serious cryptographic applications.)

* SDES encryption takes a 10 bit raw key (from which two 8 bit keys are generated as described in the
handout)
* Encrypts an 8 bit plaintext to produce an 8 bit ciphertext.

In [1]:
class SDES:
    def __init__(self):
        # 10-bit Key permutation from SDES Standard
        # Emulating       3 5 2 7 4 10 1 9 8 6
        self.__key_p10 = [2,4,1,6,3,9,0,8,7,5]

        # 08-bit Key permutation from SDES Standard
        # Emulating      6 3 7 4 8 5 10 9
        self.__key_p8 = [5,2,6,3,7,4,9,8]

        # 08-bit plain text permutation from SDES Standard
        # Emulating        2 6 3 1 4 8 5 7
        self.__text_p8  = [1,5,2,0,3,7,4,6]


        # 08-bit inverse plain text  permutation from SDES Standard
        # Emulating            4 1 3 5 7 2 8 6
        self.__text_p8_inv  = [3,0,2,4,6,1,7,5]
        
        # 08-bit expansion/permutation  plain text  permutation from SDES Standard
        # Emulating           4 1 2 3 2 3 4 1
        self.__text_ep_p8  = [3,0,1,2,1,2,3,0]

        # P4 Permutation 
        # Emulating       2 4 3 1
        self.__text_p4 = [1,3,2,0]

        self.__text_size = 8


        self.__s0 = [ [1, 0, 3, 2],
                    [3, 2, 1, 0],
                    [0, 2, 1, 3],
                    [3, 1, 3, 2]]

        self.__s1 = [[0, 1, 2, 3],
                    [2, 0, 1, 3],
                    [3, 0, 1, 0],
                    [2, 1, 0, 3]]

        self.verbose = False

        
        

    @staticmethod
    def int2bin(integer, digits):
        if integer >= 0:
            return bin(integer)[2:].zfill(digits)
        else:
            return bin(2**digits + integer)[2:]

    @staticmethod
    def permute(binVal, permutator):
        return ''.join( [binVal[i] for i  in permutator ])

    @staticmethod 
    def halfSplitVal(binVal):
        binVal_len = len(binVal)//2
        val1 = binVal[0:binVal_len] 
        val2 = binVal[binVal_len:] 
        return val1, val2
    
    @staticmethod
    def circularLeftShift(binVal,bitCount):
        return binVal[bitCount:] + binVal[0:bitCount]
        
    @staticmethod
    def bin2int(binVal):
        return int(binVal,2)

    @staticmethod
    def leftShiftBits(binVal:int,n_bits = 1):
        return binVal << n_bits

    @staticmethod
    def get_smap_number(nibble,sbox):
        row = int( nibble[0] + nibble[3] ,2)
        col = int( nibble[1] + nibble[2] ,2)
        return sbox[row][col]

    @staticmethod
    def xor(val1,val2):
        return int(val1,2) ^ int(val2,2)

    def log(self,message):
        if self.verbose:
            print(message)

    def generate_key(self,key):
        key_size = 10
        if(len(key) > key_size ):
            raise ValueError(f"Invalid key {key}({key}), it has to be 10-bit maximum")

        permuted_key = SDES.permute(key, self.__key_p10)
        self.log(f'permuted_key:{permuted_key}')

        # Divide the permuted key in 2 chunks size 5 and then perform circular left shift 
        permuted_key_1 ,permuted_key_2 = SDES.halfSplitVal(permuted_key)
        self.log(f'split Key:{permuted_key_1}, {permuted_key_2}')

        # 1-bit Circular left shift
        permuted_key_1 = SDES.circularLeftShift(permuted_key_1,1)
        permuted_key_2 = SDES.circularLeftShift(permuted_key_2,1)

        self.log(f'Circular left shift:{permuted_key_1}, {permuted_key_2}')

        # Join again split-shift 5-bit keys
        k1 = permuted_key_1 + permuted_key_2
        self.log(f'K1:{k1}')

        # Permuting now by 8-bit key permutation
        k1 = SDES.permute(k1, self.__key_p8)        
        self.log(f'Permuted K1:{k1}')

        # 2-bit Circular left shift
        permuted_key_1 = SDES.circularLeftShift(permuted_key_1,2)
        permuted_key_2 = SDES.circularLeftShift(permuted_key_2,2)
        self.log(f'Circular left shift:{permuted_key_1}, {permuted_key_2}')

        # Join again split-shift 5-bit keys
        k2 = permuted_key_1 + permuted_key_2
        # Permuting now by 8-bit key permutation
        k2 = SDES.permute(k2, self.__key_p8)
        self.log(f'Permuted K2:{k2}')

        return k1,k2

    def f_complex(self, permuted_text, key):
        # 'l' and 'r' Stand both fro left and right most significant bits
        l , r = SDES.halfSplitVal(permuted_text)
        self.log(f'Split texts are  {l}, {r}')
        
        r_permuted = SDES.permute(r,self.__text_ep_p8)
        self.log(f'E/P permuted text is {r_permuted}')

        # Xor Function from R with K
        r_xor_k_num = SDES.xor(r_permuted,key) 
        r_xor_k = SDES.int2bin(r_xor_k_num,self.__text_size)
        self.log(f'XOR from permuted and k1 {r_permuted} {key} = {r_xor_k}')

        # Split Xor Resut by half
        r_xor_k_1,r_xor_k_2  = SDES.halfSplitVal(r_xor_k)
        self.log(f'Split by half: {r_xor_k_1} , {r_xor_k_2}')

        # Split nibble will now point to the S list
        # Coordinates in the sence of: [ a b  c  d ]
        # Where 
        # row = a d
        # col = b c       
        l_s0 = SDES.get_smap_number(r_xor_k_1,self.__s0)
        r_s1 = SDES.get_smap_number(r_xor_k_2,self.__s1)
        l_s0_bin = SDES.int2bin(l_s0,2)
        r_s1_bin = SDES.int2bin(r_s1,2)
        self.log(f'S values : {l_s0}:{l_s0_bin} , {r_s1}:{r_s1_bin}')
        s_out = l_s0_bin + r_s1_bin
        self.log(f'S out values are: {s_out}')
        #Perform final permutation for given s out value
        s_out_permuted = SDES.permute(s_out,self.__text_p4)
        self.log(f'Permuted S out is: {s_out_permuted}')
        
        s_xor_l_num = SDES.xor(s_out_permuted,l)
        s_xor_l = SDES.int2bin(s_xor_l_num,4)
        self.log(f'Xor of  {l} and {s_out_permuted} is {s_xor_l}')
        # Then swith nibble positions from placese where r goes to the first position and l to the right
        return s_xor_l, r
   
    def encrypt(self, text, key):
        #text_bin = SDES.int2bin(text,self.__text_size)
        if(len(text) > self.__text_size ):
            raise ValueError(f"Invalid text size {text}, it has to be 8-bit maximum")
        k1,k2 = self.generate_key(key)

        permuted_text = SDES.permute(text,self.__text_p8)
        self.log(f'Initial permutation of {text}: {permuted_text}')

        l,r  = self.f_complex(permuted_text,k1)
        out_val =  r + l
        self.log(f'out val 1st iteration is {out_val}')
        self.log(f'-------------')
        l,r = out_val = self.f_complex(out_val,k2)
        self.log(f'-------------')
        out_val =   l + r
        self.log(f'out val 2nd iteration is {out_val}')
        ip_inv = SDES.permute(out_val,self.__text_p8_inv)
        self.log(f'IP Inverse value is {ip_inv}')
        return ip_inv

    def decrypt(self,text,key):
        #text_bin = SDES.int2bin(ciphText,self.__text_size)
        if(len(text) > self.__text_size ):
            raise ValueError(f"Invalid text size {text}, it has to be 8-bit maximum")
        k1,k2 = self.generate_key(key)

        permuted_text = SDES.permute(text,self.__text_p8)
        self.log(f'Initial permutation of {text}: {permuted_text}')

        l,r  = self.f_complex(permuted_text,k2)
        out_val =  r + l
        self.log(f'out val 1st iteration is {out_val}')
        self.log(f'-------------')
        l,r = out_val = self.f_complex(out_val,k1)
        self.log(f'-------------')
        out_val =   l + r
        self.log(f'out val 2nd iteration is {out_val}')
        ip_inv = SDES.permute(out_val,self.__text_p8_inv)
        self.log(f'IP Inverse value is {ip_inv}')
        return ip_inv

    def encrypt3SDES(self,text,k1,k2):
        cipher = self.encrypt(text,k1)
        cipher = self.decrypt(cipher,k2)
        cipher = self.encrypt(cipher,k1)
        return cipher

    def decrypt3SDES(self,text,k1,k2):
        decipher = self.decrypt(text,k1)
        decipher = self.encrypt(decipher,k2)
        decipher = self.decrypt(decipher,k1)
        return decipher


    def crack(self,cipher):
        key = 0
        text = 0
        key_len = 2**10
        text_len = 2**8
        potential_keys = []
        while key < key_len:
            while text < text_len:
                key_str = self.int2bin(key,10)
                text_str = self.int2bin(text,8)
                p_cipher = self.encrypt(text_str,key_str)
                if p_cipher == cipher:
                   potential_keys.append(key)
                key +=1
                text +=1
        return potential_keys



    

### Tests
* To verify that your implementation of SDES is correct, try the following test cases:

| Raw Key    | PlainText  | CipherText  |
| ---        | --- | --- |
| 0000000000 | 10101010  | 00010001  |
| 1110001110 | 10101010  | 11001010  |
| 1110001110 | 01010101  | 01110000  |
| 1111111111 | 10101010  | 00000100  |
---


In [2]:
keys = ['0000000000',
        '1110001110',
        '1110001110',
        '1111111111']

texts = ['10101010',
        '10101010',
        '01010101',
        '10101010']

ciphers = [
    '00010001',
    '11001010',
    '01110000',
    '00000100']


sdes = SDES()
for i in range(len(keys)):
    cipher = sdes.encrypt(texts[i],keys[i])
    print(f'{keys[i]} {texts[i]}  = {cipher} == {ciphers[i]}?  {cipher == ciphers[i]}')

0000000000 10101010  = 00010001 == 00010001?  True
1110001110 10101010  = 11001010 == 11001010?  True
1110001110 01010101  = 01110000 == 01110000?  True
1111111111 10101010  = 00000100 == 00000100?  True


In [3]:
keys = [
    '0000000000',
    '0000011111',
    '0010011111',
    '0010011111',
    '1111111111',
    '1111111111',
    '1000101110',
    '1000101110']

texts = [
    '10101010',
    '11111100',
    '10100101',
    '01010101',
    None,
    None,
    None,
    None]

ciphers = [
    None,
    None,
    None,
    None,
    '00001111',
    '01000011',
    '00011100',
    '11000010']

for i in range(len(keys)):
    if texts[i] is None:
        texts[i] = sdes.decrypt(ciphers[i],keys[i])
    else:
        ciphers[i] = sdes.encrypt(texts[i],keys[i])
print('Raw key\tPlainText\tCipherText')
for i in range(len(keys)):
    print(f'{keys[i]}\t{texts[i]}\t{ciphers[i]}')


Raw key	PlainText	CipherText
0000000000	10101010	00010001
0000011111	11111100	10011101
0010011111	10100101	10010000
0010011111	01010101	11001100
1111111111	11111111	00001111
1111111111	01100001	01000011
1000101110	00111000	00011100
1000101110	00001100	11000010


## Task1.
Use your implementation to complete the following table:

| Raw Key    | PlainText  | CipherText |
| ---        |:---:       |:---:       |
| 0000000000 | 10101010  | 00010001  |
| 0000011111 | 11111100  | 10011101  |
| 0010011111 | 10100101  | 10010000  |
| 0010011111 | 01010101  | 11001100  |
| 1111111111 | 11111111  | 00001111  |
| 1111111111 | 01100001  | 01000011  |
| 1000101110 | 00111000  | 00011100  |
| 1000101110 | 00001100  | 11000010  |

---

The DES algorithm uses keys of length 56 bits, which, when DES was originally designed, was thought
to be secure enough to meet most needs. However, due to Moores law, the increase in computing
power makes it more tractible to brute-force crack a 56-bit key. Thus, an alternative version of DES
using longer keys was desirable. The result, known as Triple DES uses two 56-bit raw keys k1 and k2
and is implemented by composing DES with itself three times in the following way

$Enc_{3DES}(p)=Enc_{DES}(k_1,Dec_{DES}(k_2,Enc_{DES}(k_1,p)))$

Here, p is the paintext to encrypt, $Enc_{DES}$ is the usuarl DES encryption algorithm and $Dec_{DES}$ is the DES encryption algorithm. This strategy doubles the number of bits in the key, at the expense of performing three times as many calculations.

The tripleDES decryption algorithm is just the reverse:

$Dec_{3DES(c)}=Dec_{DES}(k_1,Enc_{DES}(k_2,Dec_{DES}(k_1,c)))$


In [4]:
keys = [
['1000101110','0110101110'],
['1000101110','0110101110'],
['1111111111','1111111111'],
['0000000000','0000000000'],
['1000010110','0110101110'],
['1011101111','0110101110'],
['1111111111','1111111111'],
['0000000000','0000000000']]

texts = [
    '11010111',
    '10101010',
    '00000000',
    '01010010',
    None,
    None,
    None,
    None]


ciphers = [
    None,
    None,
    None,
    None,
    '11100110',
    '01010000',
    '00000100',
    '11110000']

for i in range(len(keys)):
    if texts[i] is None:
        texts[i] = sdes.decrypt3SDES(ciphers[i],keys[i][0],keys[i][1])
    else:
        ciphers[i] = sdes.encrypt3SDES(texts[i],keys[i][0],keys[i][1])
print('Raw key 1\tRaw key 2\tPlainText\tCipherText')
for i in range(len(keys)):
    print(f'{keys[i][0]}\t{keys[i][1]}\t{texts[i]}\t{ciphers[i]}')




Raw key 1	Raw key 2	PlainText	CipherText
1000101110	0110101110	11010111	10111001
1000101110	0110101110	10101010	11100100
1111111111	1111111111	00000000	11101011
0000000000	0000000000	01010010	10000000
1000010110	0110101110	11111100	11100110
1011101111	0110101110	01001111	01010000
1111111111	1111111111	10101010	00000100
0000000000	0000000000	00000000	11110000


## Task2.
Implement a class called TripleSDES and complete thefollowing table:

| Raw Key  1  | Raw Key 2  | PlainText |CipherText |
| ---        |:---:       |:---:       |:---:       |
| 1000101110 | 0110101110  | 11010111 |    10111001     |
| 1000101110 | 0110101110  | 10101010 |    11100100     |
| 1111111111 | 1111111111  | 00000000 |    11101011     |
| 0000000000 | 0000000000  | 01010010 |    10000000     |
| 1000010110 | 0110101110  | 11111100 |    11100110     |
| 1011101111 | 0110101110  | 01001111 |    01010000     |
| 1111111111 | 1111111111  | 10101010 |    00000100     |
| 0000000000 | 0000000000  | 00000000 |    11110000     |

---


## Task3.
#### Cracking SDES and TripleSDES

* The message in the file cxt1.txt was encoded using SDES. Decrypt it, and find the 10-bit raw
key used for its encryption.

* The mesage in the file cxt2.txt was encoded using TripleSDES. Decrypt it, and find the two 10-
bit raw keys used for its encryption.

**Hints**: The ciphertexts are obtained by encrypting the binary string converted from clear message with
the standard ASCII code.


In [86]:
def split_string(string,chunk_size):
    str_len = len(string)
    chunks = str_len//chunk_size
    retval = []
    for i in range(chunks):
        chunk = string[i*chunk_size:i*chunk_size + chunk_size]
        retval.append(chunk)
    offset = str_len % chunk_size
    if offset > 0:
        retval.append(string[-offset:])
    return retval

def readFile(path):
    with  open(path) as txt1:
        txt = ''.join(txt1.readlines())
        txt = txt.replace('\n','')
        txt_list = split_string(txt,8)
    return txt_list

def get_chunk_text_frequency(txt_list):
    chunk_dict = {}
    for item in txt_list:
        chunk_dict[item] = 1 if item not in chunk_dict else chunk_dict[item] + 1
    chunk_dict = {k: v for k, v in sorted(chunk_dict.items(), key=lambda item: item[1],reverse=True)}
    return chunk_dict

# Le'ts try to find the suitable cipher by tyring to crack the code given the most frequent words.
eng_word_freq = ['E','T','A','O','I','N','S','R','H','D','L','U','C','M','F','Y','W','G','P','B','V','K','X','Q','J','Z']
    

Let's assume that the first 4 top bytes are somehow representing the letter 'e' or 'E', therefore we can run a brute-force algorithm to try to find the potential keys which given such letter give the expected result.

In [118]:
sdes = SDES()
txt = readFile('ctx1.txt')
chunks = get_chunk_text_frequency(txt)
print(chunks)

{'00000001': 8, '01101110': 8, '01000111': 6, '01001111': 5, '10101111': 5, '10001000': 4, '01110100': 3, '01010111': 3, '10111010': 3, '01001100': 3, '10010111': 3, '11001101': 2, '10010000': 2, '01000000': 1, '11001011': 1, '00001001': 1, '01001010': 1, '00110010': 1}


In [121]:

e_ascii = ['e','E']
e_bin = [sdes.int2bin(ord(e),8) for e in e_ascii]
 #['00000001', '01101110', '01000111', '01001111', '10101111']
candidate_e = list(chunks.keys())[:5]
last_key = 1024 # int('1111111111',2)
decipher_text = {}
for key in range(last_key):
    key_bin = sdes.int2bin(key,10)
    for c_e in candidate_e:
        decipher_e = sdes.decrypt(c_e,key_bin)

        if decipher_e in e_bin:
            e = chr(int(decipher_e,2))
            dt = []
            is_candidate =  True
            for txt_item in txt:
                decipher = sdes.decrypt(txt_item,key_bin)
                decipher_int = int(decipher,2)
                # let's assume that encoded text consists of pure alphabetical ascii letters from 'A'(65) TO 'z' 122B
                if decipher_int > 122:
                    is_candidate = False
                decipher_chr = chr(decipher_int)
                dt.append(decipher_chr)
            if is_candidate:
                decipher_text[key_bin] = ''.join(dt).replace('\n',' ')
                print(f'Candidate key {key_bin}  => {decipher_text[key_bin]}')
    key +=1



Candidate key 1111101010  => simplifieddesisnotsecureenoughtoprovideyousufficientsecurity


Key found for txt1 : **1111101110**  decrypted text: 

> simplifieddesisnotsecureenoughtoprovideyousufficientsecurity

#### Trying now for ctx2.txt

In [123]:
txt = readFile('ctx2.txt')
chunks = get_chunk_text_frequency(txt)
print(chunks)
e_ascii = ['e','E']
e_bin = [sdes.int2bin(ord(e),8) for e in e_ascii]
 #['00000001', '01101110', '01000111', '01001111', '10101111']
candidate_e = list(chunks.keys())[:5]
last_key = 1024 # int('1111111111',2)
decipher_text = {}
for k1 in range(last_key):
    k1_bin = sdes.int2bin(k1,10)
    for k2 in range(last_key):
        k2_bin = sdes.int2bin(k2,10)
        for c_e in candidate_e:
            decipher_e = sdes.decrypt3SDES(c_e,k1_bin,k2_bin)

            if decipher_e in e_bin:
                is_candidate = True
                e = chr(int(decipher_e,2))
                dt = []
                for txt_item in txt:
                    decipher = sdes.decrypt3SDES(txt_item,k1_bin,k2_bin)
                    # let's assume that encoded text consists of pure alphabetical ascii letters from 'A'(65) TO 'z' 122B
                    decipher_int = int(decipher,2)
                    if decipher_int > 122:
                        is_candidate = False
                        break;
                    decipher_chr = chr(decipher_int)
                    dt.append(decipher_chr)
                if is_candidate:
                    decipher_text[(k1_bin,k2_bin)] = ''.join(dt).replace('\n',' ')
                    print(f'Candidate key [{k1_bin},{k2_bin}]  => {decipher_text[(k1_bin,k2_bin)]}')
        k2+=1
    k1 +=1

{'10100111': 8, '10011100': 8, '00000001': 6, '10100001': 5, '01111110': 5, '11011010': 4, '11010111': 3, '01110100': 3, '10011001': 3, '11101111': 3, '00100100': 3, '11000110': 2, '01000001': 2, '00110010': 1, '01100100': 1, '10100000': 1, '10110011': 1, '00100011': 1}
Candidate key [1111101010,0101011111]  => simplifieddesisnotsecureenoughtoprovideyousufficientsecurity


Key found for txt1 :  **\[ 1111101010,0101011111 \]** decrypted text:

>> simplifieddesisnotsecureenoughtoprovideyousufficientsecurity

# Task4.

Create a simple webserver which already stores two Raw keys (1000101110 - 0110101110) and can decrypt any ciphertext coming to it by that key using TripleSDES which you have already created and show the result in the browser. This is a simple end to end encryption. How strong is the security in this type of communication?

**Example:**

Browser input (bits are just for demo) :
http://localhost:5000/index.js?cipher=1011011010111011111100101111011100010111
Output : Hello